In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO) 
# Set to INFO for tracking training, default is WARN 

print("Using TensorFlow version %s" % (tf.__version__)) 
# This notebook is intended for tested for TF 1.3

CATEGORICAL_COLUMNS = ["user_id", "order_id", "product_id", "aisle_id", "department_id"]

# Columns of the input csv file
COLUMNS = ["user_id", "user_orders", "order_id", "order_number", "order_dow", "order_hour_of_day", 
           "days_since_prior_order", "product_id", "add_to_cart_order", "reordered", "aisle_id", "department_id"]
# "add_to_cart_order",

FEATURE_COLUMNS = ["user_id", "user_orders", "order_id", "order_number", "order_dow", "order_hour_of_day", 
           "days_since_prior_order", "product_id", "add_to_cart_order", "aisle_id", "department_id"]

# # Feature columns for input into the model
# FEATURE_COLUMNS = ["age", "workclass", "education", 
#                    "education_num", "marital_status",
#                    "occupation", "relationship", "race", 
#                    "gender", "capital_gain", "capital_loss",
#                    "hours_per_week", "native_country"]

Using TensorFlow version 1.5.0


In [2]:
import pandas as pd

# df = pd.read_csv("train53w6.csv", header=None, names=COLUMNS)
#                  dtype={"user_id":str, "user_orders":str, "order_id":str, "order_number":str,
#                  "order_dow":str, "order_hour_of_day":str,"days_since_prior_order":str, 
#                  "product_id":str, "add_to_cart_order":str, "reordered":str,
#                  "aisle_id":str, "department_id":str})

In [114]:
df.shape

(966804, 12)

In [113]:
df.head()

,user_id,user_orders,order_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id
0,27u,81,361493o,1,3w,9h,30s,25718p,1c,0,26a,7d
1,27u,81,1007361o,5,2w,15h,4s,25718p,1c,1,26a,7d
2,27u,81,3141790o,11,5w,11h,1s,25718p,1c,1,26a,7d
3,27u,81,1380784o,12,1w,11h,3s,25718p,1c,1,26a,7d
4,27u,81,1659155o,13,2w,17h,1s,25718p,1c,1,26a,7d


In [6]:
df.loc[7048364] = "a"

In [11]:
df.user_id.dtype

dtype('O')

In [124]:
df.user_id.nunique()

9166

In [12]:
df.reordered.dtype

dtype('O')

In [9]:
df[-1,1] = 1

In [13]:
df.user_orders.dtype

dtype('O')

In [10]:
df[-1,3] = 1

In [14]:
df[-1,-3] = 1

In [83]:
type(df.order_id)

pandas.core.series.Series

In [64]:
df['user_orders'] = df['user_orders'].astype(int)

In [129]:
df.add_to_cart_order.dtype

dtype('int64')

In [3]:
df.describe(include=[np.number])

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,16278.000000,1.627800e+04,16278.000000,16278.000000,16278.000000,16278.000000
mean,38.767416,1.894312e+05,10.072368,1081.769382,87.915469,40.390466
std,13.850370,1.057114e+05,2.567474,7584.547894,403.140665,12.479308
min,17.000000,1.349200e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.167385e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.778295e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.383840e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,3770.000000,99.000000


In [192]:
df.aisle_id.dtype

dtype('O')

In [4]:
df.describe(include=[np.object])

,workclass,education,marital_status,occupation,relationship,race,gender,native_country,income_bracket
count,16278,16278,16278,16278,16278,16278,16278,16278,16278
unique,9,16,7,15,6,5,2,41,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,11208,5283,7401,2031,6521,13944,10857,14659,12433


In [7]:
df.corr()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
age,1.000000,-0.076556,0.019944,0.076362,0.055304,0.077096
fnlwgt,-0.076556,1.000000,-0.029951,-0.011705,0.007396,-0.003234
education_num,0.019944,-0.029951,1.000000,0.130089,0.083133,0.134766
capital_gain,0.076362,-0.011705,0.130089,1.000000,-0.031106,0.089421
capital_loss,0.055304,0.007396,0.083133,-0.031106,1.000000,0.054926
hours_per_week,0.077096,-0.003234,0.134766,0.089421,0.054926,1.000000


In [3]:
BATCH_SIZE = 30

def generate_input_fn(filename, num_epochs=None, shuffle=True, batch_size=BATCH_SIZE):
    df = pd.read_csv(filename, header=None, names=COLUMNS)
    labels = df["reordered"].astype(int) #.apply(lambda x: ">50K" in x).astype(int)
#     del df["user_orders"] # Unused column
#     del df["add_to_cart_order"]
#     del df["aisle_id"]
#     del df["department_id"]
#     del df["order_id"]
#     del df["days_since_prior_order"]
#     del df["order_number"]
    del df["reordered"] # Labels column, already saved to labels variable
    
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        y=labels,
        batch_size=batch_size,
        num_epochs=3,
        shuffle=shuffle)

print('input function configured')

input function configured


In [138]:
df.reordered.dtype

dtype('O')

In [7]:
%%time
# The layers module contains many utilities for creating feature columns.

# Categorical base columns.
# order_dow = tf.feature_column.categorical_column_with_vocabulary_list(key="order_dow", 
#                                                                    vocabulary_list=["0", "1", "2", "3", "4", "5", "6"])
# race = tf.feature_column.categorical_column_with_vocabulary_list(key="race",
#                                                                  vocabulary_list=["Amer-Indian-Eskimo",
#                                                                        "Asian-Pac-Islander",
#                                                                        "Black", "Other",
#                                                                        "White"])

# order_hour_of_day = tf.feature_column.categorical_column_with_hash_bucket(
#   "order_hour_of_day", hash_bucket_size=100)
aisle_id = tf.feature_column.categorical_column_with_hash_bucket(
  "aisle_id", hash_bucket_size=100)
department_id = tf.feature_column.categorical_column_with_hash_bucket(
  "department_id", hash_bucket_size=100)
order_id = tf.feature_column.categorical_column_with_hash_bucket(
  "order_id", hash_bucket_size=2000000)
user_id = tf.feature_column.categorical_column_with_hash_bucket(
  "user_id", hash_bucket_size=50000)
product_id = tf.feature_column.categorical_column_with_hash_bucket(
  "product_id", hash_bucket_size=50000)
# days_since_prior_order = tf.feature_column.categorical_column_with_hash_bucket(
#   "days_since_prior_order", hash_bucket_size=100)
# add_to_cart_order = tf.feature_column.categorical_column_with_hash_bucket(
#   "add_to_cart_order", hash_bucket_size=100)
# order_number = tf.feature_column.categorical_column_with_hash_bucket(
#   "order_number", hash_bucket_size=500)

print('Categorical columns configured')

Categorical columns configured
Wall time: 0 ns


In [190]:
aisle_id.dtype

tf.string

In [8]:
%%time
# Continuous base columns.
user_orders = tf.feature_column.numeric_column("user_orders")
order_number = tf.feature_column.numeric_column("order_number")
add_to_cart_order = tf.feature_column.numeric_column("add_to_cart_order")
days_since_prior_order = tf.feature_column.numeric_column("days_since_prior_order")
order_hour_of_day = tf.feature_column.numeric_column("order_hour_of_day")
order_dow = tf.feature_column.numeric_column("order_dow")
# # capital_loss  = tf.feature_column.numeric_column("capital_loss")
# hours_per_week = tf.feature_column.numeric_column("hours_per_week")

print('Continuous columns configured')

Continuous columns configured
Wall time: 0 ns


In [16]:
user_orders.dtype

tf.float32

In [11]:
# # Transformations.

order_hour_of_day_buckets = tf.feature_column.bucketized_column(
    order_hour_of_day, boundaries=[ 0, 6, 12, 18 ])
order_day_hour = tf.feature_column.crossed_column(
    ["order_hour_of_day", "order_dow"], hash_bucket_size=int(1e4))

    
# age_buckets = tf.feature_column.bucketized_column(
#     age, boundaries=[ 18, 25, 30, 35, 40, 45, 50, 55, 60, 65 ])

# education_occupation = tf.feature_column.crossed_column(
#     ["education", "occupation"], hash_bucket_size=int(1e4))

# age_race_occupation = tf.feature_column.crossed_column(
#     [age_buckets, "race", "occupation"], hash_bucket_size=int(1e6))

# country_occupation = tf.feature_column.crossed_column(
#     ["native_country", "occupation"], hash_bucket_size=int(1e4))

# print('Transformations complete')

In [13]:
%%time
# Wide columns and deep columns.
wide_columns = [order_dow, order_hour_of_day, aisle_id,
      user_id, department_id, order_id, add_to_cart_order,
      product_id, days_since_prior_order, order_day_hour, order_hour_of_day_buckets]
      #add_to_cart_order,user_orders]

deep_columns = [
    # Multi-hot indicator columns for columns with fewer possibilities
#     tf.feature_column.indicator_column(order_dow),
#     tf.feature_column.indicator_column(order_hour_of_day),
    tf.feature_column.embedding_column(aisle_id, dimension=30),
    tf.feature_column.embedding_column(department_id, dimension=30),
#     tf.feature_column.indicator_column(order_id),
#     tf.feature_column.indicator_column(days_since_prior_order),
#     tf.feature_column.indicator_column(add_to_cart_order),
#     tf.feature_column.indicator_column(order_number),
#     tf.feature_column.indicator_column(department_id),
    # Embeddings for categories with more possibilities
    tf.feature_column.embedding_column(order_id, dimension=30),
    tf.feature_column.embedding_column(user_id, dimension=30),
    tf.feature_column.embedding_column(product_id, dimension=30),
    # Numerical columns
    order_number,
    add_to_cart_order,
    days_since_prior_order,
    order_hour_of_day,
    order_dow,
    user_orders,
]

print('wide and deep columns configured')

wide and deep columns configured
Wall time: 2.5 ms


In [14]:
%%time
def create_model_dir(model_type):
    return 'models/model_' + model_type + '_' + str(int(time.time()))

# If new_model=False, pass in the desired model_dir 
def get_model(model_type, new_model=False, model_dir=None):
    if new_model or model_dir is None:
        model_dir = create_model_dir(model_type) # Comment out this line to continue training a existing model
    print("Model directory = %s" % model_dir)
    
    m = None
    
    # Linear Classifier
    if model_type == 'WIDE':
        m = tf.estimator.LinearClassifier(
            model_dir=model_dir, 
            feature_columns=wide_columns)

    # Deep Neural Net Classifier
    if model_type == 'DEEP':
        m = tf.estimator.DNNClassifier(
            model_dir=model_dir,
            feature_columns=deep_columns,
            hidden_units=[200, 150])

    # Combined Linear and Deep Classifier
    if model_type == 'WIDE_AND_DEEP':
        m = tf.estimator.DNNLinearCombinedClassifier(
                model_dir=model_dir,
                linear_feature_columns=wide_columns,
                dnn_feature_columns=deep_columns,
                dnn_hidden_units=[200, 150])
        
    print('estimator built')
    
    return m, model_dir
    
MODEL_TYPE = 'WIDE_AND_DEEP'
model_dir = create_model_dir(model_type=MODEL_TYPE)
m, model_dir = get_model(model_type = MODEL_TYPE, model_dir=model_dir)

Model directory = models/model_WIDE_AND_DEEP_1518615108
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/model_WIDE_AND_DEEP_1518615108', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000000013AA4F60>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
estimator built
Wall time: 10 ms


In [15]:
%%time 

train_file = str("train53w6.csv") 
# "gs://cloudml-public/census/data/adult.data.csv"
# storage.googleapis.com/cloudml-public/census/data/adult.data.csv

m.train(input_fn=generate_input_fn(train_file), 
      steps=200)

print('training done')

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into models/model_WIDE_AND_DEEP_1518615108\model.ckpt.
INFO:tensorflow:loss = 33.97393, step = 1
INFO:tensorflow:global_step/sec: 125.784
INFO:tensorflow:loss = 14.251782, step = 101 (0.800 sec)
INFO:tensorflow:Saving checkpoints for 200 into models/model_WIDE_AND_DEEP_1518615108\model.ckpt.
INFO:tensorflow:Loss for final step: 13.408324.
training done
Wall time: 30.8 s


In [16]:
%%time
test_file  = str("test53w6.csv") 

results = m.evaluate(input_fn=generate_input_fn(test_file, num_epochs=3, shuffle=True), 
                     steps=100)
print('evaluate done')
print('\nAccuracy: %s' % results['accuracy'])

INFO:tensorflow:Starting evaluation at 2018-02-14-13:36:42
INFO:tensorflow:Restoring parameters from models/model_WIDE_AND_DEEP_1518615108\model.ckpt-200
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-02-14-13:36:45
INFO:tensorflow:Saving dict for global step 200: accuracy = 0.84533334, accuracy_baseline = 0.84533334, auc = 0.53889465, auc_precision_recall = 0.86502326, average_loss = 0.4432231, global_step = 200, label/mean = 0.84533334, loss = 13.296693, prediction/mean = 0.88115406
evaluate done

Accuracy: 0.84533334
Wall time: 9.97 s


In [ ]:
# Create a dataframe to wrap in an input function
df = pd.read_csv("test53w6.csv", header=None, names=COLUMNS)
data_predict = df.iloc[8000:8005]
data_predict.head() # show this before deleting, so we know what the labels are

# If you run this cell twice, it will give an error since you'd be deleting something that was already gone
# del data_predict["fnlwgt"] # Unused column
# del data_predict["reordered"] # remove the label column

predict_input_fn = tf.estimator.inputs.pandas_input_fn(
        x=data_predict,
        batch_size=1,
        num_epochs=1,
        shuffle=False)

predictions = m.predict(input_fn=predict_input_fn)

for prediction in predictions:
    print("Predictions:    {} with probabilities {}\n".format(prediction["classes"], prediction["probabilities"]))

In [17]:
def column_to_dtype(column):
    if column in CATEGORICAL_COLUMNS:
        return tf.string
    else:
        return tf.float32
    
feature_spec = {
    column: tf.FixedLenFeature(shape=[1], dtype=column_to_dtype(column))
        for column in FEATURE_COLUMNS
}
serving_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)
m.export_savedmodel(export_dir_base=model_dir,# + '/export/', 
                            serving_input_receiver_fn=serving_fn)

INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Restoring parameters from models/model_WIDE_AND_DEEP_1518615108\model.ckpt-200
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"models/model_WIDE_AND_DEEP_1518615108\\temp-b'1518615455'\\saved_model.pb"


b'models/model_WIDE_AND_DEEP_1518615108\\1518615455'